In [40]:
# Gamaliel Marines Olvera - A01708746
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np

import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random

In [41]:
class CleaningAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.cleaning_step = 0

    def move(self):
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=False)
        new_position = self.random.choice(neighbors)

        if not self.model.grid.is_cell_empty(new_position):
            return
        self.model.grid.move_agent(self, new_position)

    def step(self):
        if self.model.is_dirty(self.pos):
            self.model.is_clean(self.pos)
            self.cleaning_step += 1
        else:
            self.move()

In [42]:
class CleaningModel(Model):
    def __init__(self, width, height, num_agents, dirty_cells, max_steps):
        self.num_agents = num_agents
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.running = True
        self.steps = 0
        self.max_steps = max_steps
        self.total_cells = width * height
        self.clean_cells = 0
        self.dirt = np.zeros((width, height))
        total_dirty_cells = int(dirty_cells * self.total_cells)

        all_cells = [(x, y) for x in range(width) for y in range(height)]

        np.random.shuffle(all_cells)
        dirty_cells = all_cells[:total_dirty_cells]

        for cell in dirty_cells:
            self.dirt[cell[0]][cell[1]] = 1

        for i in range(self.num_agents):
            x, y = np.random.choice(range(width)), np.random.choice(range(height))
            agent = CleaningAgent(i, self)
            self.grid.place_agent(agent, (x, y))
            self.schedule.add(agent)

        self.datacollector = DataCollector({"Dirt": lambda m: np.sum(m.dirt)})


In [43]:
def run_cleaning_simulation(width, height, num_agents, dirty_cells, max_steps):
    model = CleaningModel(width, height, num_agents, dirty_cells, max_steps)
    for step in range(max_steps):
        model.step()
        if np.sum(model.dirt) == 0:
            return step

    return max_steps

In [44]:
WIDTH = 100
HEIGHT = 100
NUM_AGENTS = 1
DIRTY_CELLS = 0.1  # Porcentaje de celdas sucias
MAX_GENERATIONS = 100

start_time = time.time()
result = run_cleaning_simulation(WIDTH, HEIGHT, NUM_AGENTS, DIRTY_CELLS, MAX_GENERATIONS)
total_time = time.time() - start_time

# Imprimimos el tiempo que le tomó correr al modelo y el número de pasos al final
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=total_time)))
print("Steps:", result)

Tiempo de ejecución: 0:00:00.013058
Steps: 100
